# 1. 결측치 detection

In [65]:
raw=pd.read_csv('./train.csv')

## 참고: mean(), std() 연산 시 : NaN은 분석 대상(population)에서 제외

In [66]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [68]:
display(raw.isnull().sum())
raw.isnull()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,False,False,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
886,False,False,False,False,False,False,False,False,False,False,True,False
887,False,False,False,False,False,False,False,False,False,False,False,False
888,False,False,False,False,False,True,False,False,False,False,True,False
889,False,False,False,False,False,False,False,False,False,False,False,False


# 2. 결측치 처리(Imputation)

결측치 처리 방법으로는 크게 Single Imputation, Multiple Imputation으로 나뉜다.

#### - Single Imputation
    
    1) hot deck: 최빈값으로 대체
    
    2) mean imputation: 평균값으로 대체
    
    3) regression imputation: 회귀추정값으로 대체
    
        - 결측대체값에 변동이 전혀 없는 fitted value 를 넣다보니 계수추정치에 대한 신뢰도가 과대평가되는 경향이 있다.
        
        - 이를 방지하기 위해 residual 을 추가해준 것을 stochastic regression imputation 이라고 부르며, single imputation 
          중에서는 편향이 제일 적은 결과를 가져온다고 알려져 있다.


    
    
#### - Multiple Imputation

## 1) Mean값 활용

In [5]:
import pandas as pd
from sklearn.impute import SimpleImputer

In [57]:
raw=pd.read_csv('./train.csv')
raw=raw.drop('Name',axis=1)
raw=raw.drop('Sex',axis=1)
raw=raw.drop('Cabin',axis=1)
raw=raw.drop('Ticket',axis=1)
raw=raw.drop('Embarked',axis=1)
raw

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,1,0,3,22.0,1,0,7.2500
1,2,1,1,38.0,1,0,71.2833
2,3,1,3,26.0,0,0,7.9250
3,4,1,1,35.0,1,0,53.1000
4,5,0,3,35.0,0,0,8.0500
...,...,...,...,...,...,...,...
886,887,0,2,27.0,0,0,13.0000
887,888,1,1,19.0,0,0,30.0000
888,889,0,3,NaN,1,2,23.4500
889,890,1,1,26.0,0,0,30.0000


In [59]:
no_null=raw.copy()
cols_with_missing = [col for col in no_null.columns if no_null[col].isnull().any()]

#결측치 대체 여부 표시
for col in cols_with_missing:
    no_null[col+'_was_missing']=no_null[col].isnull()
    
imputer=SimpleImputer()
no_null=pd.DataFrame(imputer.fit_transform(no_null))

In [62]:
raw.info()
no_null.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Age          714 non-null    float64
 4   SibSp        891 non-null    int64  
 5   Parch        891 non-null    int64  
 6   Fare         891 non-null    float64
dtypes: float64(2), int64(5)
memory usage: 48.9 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       891 non-null    float64
 1   1       891 non-null    float64
 2   2       891 non-null    float64
 3   3       891 non-null    float64
 4   4       891 non-null    float64
 5   5       891 non-null    float64
 6   6       891 non-null    float64
 7   7       891 non-null    f

## 2) 그룹별 Mean 활용

In [ ]:
df['value'] = df['value'].fillna(df.groupby('name')['value'].transform('mean'))

# 2. 최빈값 활용

- 문자형으로 표현됐을 경우, Mean 사용 불가 -> 최빈값/Drop 활용

In [63]:
raw=pd.read_csv('./train.csv')

no_null=raw.copy()
cols_with_missing = [col for col in no_null.columns if no_null[col].isnull().any()]

#결측치 대체 여부 표시
for col in cols_with_missing:
    no_null[col+'_was_missing']=no_null[col].isnull()
    
imputer=SimpleImputer(strategy="most_frequent")
no_null=pd.DataFrame(imputer.fit_transform(no_null))

In [64]:
raw.info()
no_null.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       891 non-null    object
 1   1       891 non-null    ob